# Convex hull<T: Float>
 

Exploring potential methodologies to extend the convex hull to floating point numbers. This is not necessarily straight-forward due to rounding errors, NANs etc. See [Sorting float](sorting_float.ipynb). For the implementation of convex hull using integer numbers see [Convex Hull](convex_hull.ipynb).

In [2]:
:dep num
use num::Float;
use std::cmp::Ordering;
use core::f64::{NAN, INFINITY, NEG_INFINITY};

In [3]:
#[derive(Debug, PartialEq, PartialOrd, Clone, Copy)]
struct Point {
    x: f64,
    y: f64
}

impl Point {
    fn new(x: f64, y: f64) -> Self {
        Self {x, y}
    }
}

Here the compiler is told to automatically implement traits derived using the [#[derive]](https://doc.rust-lang.org/stable/rust-by-example/trait/derive.html) attribute.
- Deriving [Debug](https://doc.rust-lang.org/std/fmt/trait.Debug.html) let's us easily print the struct through {:?} formatter
- Deriving [PartialEq](https://doc.rust-lang.org/std/cmp/trait.PartialEq.html) but for partial equivalence. Deriving [Eq](https://doc.rust-lang.org/std/cmp/trait.Eq.html) not allowed.
- [PartialOrd](https://doc.rust-lang.org/std/cmp/trait.PartialEq.html) makes our type ordered so that it can be sorted
- [Clone](https://doc.rust-lang.org/std/clone/trait.Clone.html) is required for struct to be Copy.
- [Copy](https://doc.rust-lang.org/std/marker/trait.Copy.html) because all fields are trivially and inexpensively copyable.

### Sorting lists of points

A possible way of sorting a vectors of Points as defined above is by defining a comparator to give to [sort_by](https://doc.rust-lang.org/std/primitive.slice.html#method.sort_by) function.

In [4]:
fn order_nanlast<F: Float>(a: &F, b: &F) -> Ordering {
    // Order floats by putting NaN last
    match (a, b) {
        (x, y) if x.is_nan() && y.is_nan() => Ordering::Equal,
        (x, _) if x.is_nan() => Ordering::Greater,
        (_, y) if y.is_nan() => Ordering::Less,
        (_, _) => a.partial_cmp(b).unwrap()
    }
}

fn order_points_nanlast(a: &Point, b: &Point) -> Ordering {
    // Order Point<T: Float> by putting NaNs lexicographically last
    order_nanlast(&a.x, &b.x).then(order_nanlast(&a.y, &b.y))
}

Oh and btw, assigning a reference to a point generated by new is not allowed and would produce error "temporary value dropped while borrowed". The borrow checker 

## Sorting 2d points

In [5]:
let order1 = order_points_nanlast(&Point::new(INFINITY, 0.0), &Point::new(1.0, 0.0));
let order2 = order_points_nanlast(&Point::new(0.0, 0.0), &Point::new(NAN, 0.0));
let order3 = order_points_nanlast(&Point::new(0.0, NAN), &Point::new(0.0, 0.0));
let order4 = order_points_nanlast(&Point::new(0.0, NAN), &Point::new(0.0, NAN));
println!("{:?}", order1);
println!("{:?}", order2);
println!("{:?}", order3);
println!("{:?}", order4);

Greater
Less
Greater
Equal


## The algorithm

The idea of the algorithm is to traverse the sorted list of points and simply discarding the points that does not form a right turn. The points are first sorted lexicographically and are then traversed twice; once left to right for forming the upper part of the hull and once right to left for the lower part. The two parts are then combined so that the points of the convex hull are given uniquely starting from the lexicographically lowest point and following the hull in counter-clockwise fashion.

First lets bikeshed the representation of orientation.

In [6]:
#[derive(Debug, PartialEq, Eq)]
enum Turn {
    CW,   // Clockwise
    CCW,  // Counter-clockwise
}

In [7]:
/// Returns the orientation of three consecutive points in the 2D plane.
fn orientation(a: &Point, b: &Point, c: &Point) -> Option<Turn> {
    let crossprod = (b.x - a.x)*(c.y - b.y) - (b.y - a.y)*(c.x - b.x);
    if crossprod < -1e-16 {
        Some(Turn::CW)
    } else if crossprod > 1e-16 {
        Some(Turn::CCW)
    } else {
        None
    }
}

In [10]:
println!("{:?}", orientation(&Point{x: 0.0, y: 0.0}, &Point{x: 1.0, y: 0.0}, &Point{x: 1.0, y: 1.0}));  // Should be Some(CCW)
println!("{:?}", orientation(&Point{x: 0.0, y: 0.0}, &Point{x: 0.0, y: 1.0}, &Point{x: 1.0, y: 1.0}));  // Should be Some(CW)
println!("{:?}", orientation(&Point{x: 0.0, y: 0.0}, &Point{x: 1.0, y: 1.0}, &Point{x: 2.0, y: 2.0}));  // Should be None

Some(CCW)
Some(CW)
None


In [12]:
let order1 = order_points_nanlast(&Point::new(0.0, INFINITY), &Point::new(1.0, 0.0));
let order2 = order_points_nanlast(&Point::new(0.0, 0.0), &Point::new(NAN, 0.0));
let order3 = order_points_nanlast(&Point::new(0.0, NAN), &Point::new(0.0, 0.0));
let order4 = order_points_nanlast(&Point::new(0.0, NAN), &Point::new(0.0, NAN));
println!("{:?}", order1);
println!("{:?}", order2);
println!("{:?}", order3);
println!("{:?}", order4);

Less
Less
Greater
Equal


In [13]:
/// Computes the convex hull of given points in O(nlogn)
fn convex_hull(points: &[Point]) -> Vec<Point> {
    // TODO: Actually handle NAN points: not part of CH.
    if points.len() < 2 {
        return points.to_vec();
    }
    
    let mut points = points.to_vec();
    points.sort_by(order_points_nanlast); 
    let points = points;  // Now immutable
    let n = points.len();
    
    // Traverse points to form the upper part of CH
    // TODO: Separate into separate function
    let mut upper = vec![points[0], points[1]];
    for p in points.iter().skip(2) {
        let mut pos = 0;
        while upper.len() - pos >= 2 {
            let length = upper.len();
            if let Some(Turn::CW) = orientation(&upper[length-pos-2], &upper[length-pos-1], p) {
                pos = pos + 1;  // Middle point belongs to to the CH for now
            } else {
                upper.remove(length-pos-1);  // Middle point is not on the CH boundary: remove.
            }
        }
        upper.push(*p);
    }
    
    // Traverse points to form the lower part of CH
    let mut lower = vec![points[n-1], points[n-2]];
    for p in points.iter().rev().skip(3) {
        let mut pos = 0;
        while lower.len() - pos >= 2 {
            let length = lower.len();
            if let Some(Turn::CW) = orientation(&lower[length-pos-2], &lower[length-pos-1], p) {
                pos = pos + 1;  // Middle point belongs to to the CH for now
            } else {
                lower.remove(length-pos-1);  // Middle point is not on the CH boundary: remove.
            }
        }
        lower.push(*p);
    }
    
    // Return upper + lower concatenated. Ends are duplicates and thusly removed
    [&upper[..], &lower[1..lower.len()-1]].concat()
}

### Example usage

Testing the algorithm for an example set of points

In [14]:
let points = vec![
    Point::new(3.0, 6.0),
    Point::new(5.0, 2.0),
    Point::new(4.0, 4.01),
    Point::new(2.0, 3.0),
    Point::new(1.0, 1.0),
    Point::new(4.0, 3.0),
    Point::new(3.0, 4.0),
    Point::new(0.0, 5.0),
];

let mut ch = convex_hull(&points);
ch

[Point { x: 0.0, y: 5.0 }, Point { x: 3.0, y: 6.0 }, Point { x: 4.0, y: 4.01 }, Point { x: 5.0, y: 2.0 }, Point { x: 1.0, y: 1.0 }]

In [15]:
ch.push(ch[0]);  // Little trick to make CH close on itself.

In [16]:
println!("Notice that the near colinear point is included in CH: {:?}", ch[2]);

Notice that the near colinear point is included in CH: Point { x: 4.0, y: 4.01 }


### Plot

Using [plotters](https://github.com/38/plotters) library to visually display the resulting convex hull.

In [17]:
:dep plotters = { git = "https://github.com/38/plotters", default_features = false, features = ["evcxr", "line_series", "point_series"] }
extern crate plotters;
use plotters::prelude::*;
use plotters::series::*;

In [18]:
let figure = evcxr_figure((640, 480), |root| {
    root.fill(&WHITE);
    let mut chart = ChartBuilder::on(&root)
        //.caption("Convex hull", ("Arial", 32).into_font())
        .margin(5)
        .x_label_area_size(30)
        .y_label_area_size(30)
        .build_ranged(-0f32..6f32, 0f32..6f32)?;

    chart.configure_mesh().draw()?;

    chart.draw_series(PointSeries::of_element(
        points.iter().map(|p: &Point| (p.x as f32, p.y as f32)),
        5,
        ShapeStyle::from(&RED).filled(),
        &|coord, size, style| {
            EmptyElement::at(coord)
                + Circle::new((0, 0), size, style)
                + Text::new(
                    format!("{:?}", coord),
                    (0, 15),
                    ("sans-serif", 15).into_font(),
                )
        },
    )).
    unwrap()
    .label("Input points")
    .legend(|(x,y)| Circle::new((x+10, y), 5, ShapeStyle::from(&RED).filled()));
    
    chart.draw_series(LineSeries::new(
        ch.iter().map(|p: &Point| (p.x as f32, p.y as f32)),
        &BLUE)
    ).unwrap()
     .label("Convex hull")
     .legend(|(x, y) | PathElement::new(vec![(x,y), (x + 20,y)], &BLUE));

    chart.configure_series_labels()
        .background_style(&WHITE.mix(0.8))
        .border_style(&BLACK)
        .draw()?;
    Ok(())
});
figure

1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
6.0
 
 
 
 
(3.0, 6.0)
 
 
 
(5.0, 2.0)
 
 
 
(4.0, 4.01)
 
 
 
(2.0, 3.0)
 
 
 
(1.0, 1.0)
 
 
 
(4.0, 3.0)
 
 
 
(3.0, 4.0)
 
 
 
(0.0, 5.0)
 
 
 
 
 
Input points
 
 
Convex hull